In [19]:
import keras
from keras.applications import inception_v3
from keras import backend as K
from keras.preprocessing import image
import tensorflow as tf

K.set_learning_phase(0)

In [2]:
model = inception_v3.InceptionV3(include_top=False, weights='imagenet')

In [20]:
layer_weight = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

# 각 Layer에 이름 붙이기
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [46]:
loss = K.variable(0.)

for name, w in layer_weight.items():

    outputs = layer_dict[name].output

    scaler = K.cast(K.prod(K.shape(outputs)), "float32")
    
    loss = loss + w * K.sum(K.square(outputs[:, 2:-2, 2:-2, :])) / scaler

In [50]:
gen_image = model.input

grads = K.gradients(loss, gen_image)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

fetch_loss_and_grads = K.Function([gen_image], [loss, grads])

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])  ## outputs 
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(img, iterations, step, max_loss=None):

    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(img)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

def resize_img(img, size):
    img = np.copy(img)
    factors = (1, float(size[0]) / img.shape[1], float(size[1]) / img.shape[2], 1)
    return sp.ndimage.zoom(img, factors, order=1)

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    
    return img

def deprocess_image(x):
    
    x = x.reshape((x.shape[1], x.shape[2], 3))
    
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype("uint8")
    return x


In [49]:
#### Parameters ####
K.get_session().run(tf.global_variables_initializer())
step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 30
max_loss = 10.
fname = 'result_deepdream.png'

from skimage import data

# # img = data.chelsea()
# img = sp.misc.face()
# img = image.img_to_array(img)
# img = np.expand_dims(img, axis=0)
# img = inception_v3.preprocess_input(img)

img = preprocess_image("./deepdream_parc.png")

original_shape = img.shape[1:3]
successive_shapes = [original_shape]

for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale**i)) for dim in original_shape])
    successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1] # 스케일 리스트 

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0]) # starting from smallest image

for shape in successive_shapes:
    
    print('Processing image shape', shape)
    
    img = resize_img(img, shape)
    
    img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)

    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)

    same_size_original = resize_img(original_img, shape)
    
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    
    shrunk_original_img = resize_img(original_img, shape)
    
# plt.imshow(deprocess_image(img))
# image.save_img('result_deepdream.png', deprocess_image(img))
save_img(img, fname)